<a href="https://colab.research.google.com/github/connorgrannis/nch_python_workshop/blob/master/Week4_statistics_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#__DISCLAIMER__
We are not the most equipped to go through an in depth explanation of all the statistical methods and models we will be presenting in this workshop. We will try and cover the basics, but this week's workshop is designed in mind to present to you the "toolbox" and not necessarily best use cases for each.

#__Statistics in Python__
There are a number of ways to go about planning and creating different statistical models using Python. From a simple T-test to larger and more complex models, much of what you will ever need to do (statistically) in Python is handled in just two major libraries: Scipy and Statsmodels.

#__Scipy__
Scipy is an enormous python library similar to the likes of Numpy, and provides a huge array of tools for scientific computing. This ranges from things like linear algebra to image manipulation, and each of Scipy's specialties are broken down and contained within sub-branches contained within the main package. The one we'll be focusing on is Scipy.stats. 

[link to scipy main page]

In [0]:
!pip install scipy

In [0]:
from scipy import stats

#__Statsmodels__
Statsmodels is built off scipy.stats, but contains more specific statistical models using a formula framework similar to something you might find in R. Statsmodels was also built to work very well with Pandas dataframes. 

[link to statsmodel main page]

In [0]:
!pip install statsmodels

In [0]:
import statsmodels.api as sm

#__Other Packages__

While the two libraries above will cover the bulk, if not all, of all your statistical needs there are several other packages that you may find useful. One we will talk use for some things in this week's workshop is Pingouin. Pingouin is similar to the previous packages, though slightly more limited in its scope, and offers several useful statistical models that rival in utitlity some of the pre-established versions in scipy and statmodels. One of the great things about Pingouin is how much more approachable it is than some of the previous mentioned packages. There are likely several other statistical packages in Python, but these are the three we'll be using today.

[pingouin link]

In [0]:
!pip install pingouin

In [0]:
import pingouin as pg

#__t-tests__

Lets start with a simple t-test. To begin we'll load back in the titanic dataset into a pandas dataframe just like we've been doing in previous sections.


In [0]:
import pandas as pd

titanic = pd.read_csv('https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv')

In [0]:
titanic.head() #to take a quick look at the data we're working with

So looking at the data above, let's do a basic 2-sided t-test to see if age differs between those who survived or didn't.

We'll use statsmodels and pingouin to do the same test with the same data.

Starting with Scipy. 

Here's the function we'll use: [documention](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)

In [0]:
## using Scipy.stats

from scipy import stats   # we only need the stats portion of scipy

survived = titanic['Survived'] == 1
died = titanic['Survived'] == 0

t, p = stats.ttest_ind(titanic[survived]['Age'], titanic[died]['Age'], nan_policy='omit')

print(f't-stat is: {t}\np-value is: {p}')

Let's take a closer look at what we did here.

First we imported the stats module from scipy: `from scipy import stats`

The next two lines:
```
survived = titanic['Survived'] == 1
died = titanic['Survived'] == 0
```
create a boolean 'filter' where the row is `True` if the value in the 'Survived' column matches the condition (==1 for survived, and ==0 for died). 

We can now use this filter on our main dataframe to split our data into two groups, those who survived and those who did not.

Now we have everything prepared to conduct our t-test using `stats.ttest_ind`.
We specify our two population arrays `titanic[survived]['Age']` for the ages of those that survived, and `titanic[died]['Age']` for those that did not. One final note is the last argument of the function: `nan_policy='omit'`, this is how we are asking the function to handle missing data. 

Now lets run the exact same test using statsmodels. It's very similar to how we handled it with Scipy.

[Documentation on the function](https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.ttest_ind.html)

In [0]:
# t-test using statsmodels
import statsmodels.stats as sms

survived = titanic['Survived'] == 1
died = titanic['Survived'] == 0

t, p, df = sms.weightstats.ttest_ind(titanic[survived]['Age'], titanic[died]['Age'])

print(f'''
t-stat is: {t}
p-value is: {p}
degrees of freedom: {df}
''')

Well that didn't work. That's because the statsmodels' t-test doesn't have a way to handle missing data at the t-test level. we'll need to handle it at the dataframe level instead. We can use the pandas method df.dropna() to do that.

### Handling Missing Data

In [0]:
titanic_no_missing = titanic.dropna()

# This removes the entire row containing missing data in ANY column. if we want to only remove rows with missing data in the 'Age' column
titanic_no_age_missing = titanic.dropna(subset=['Age'])


The subset can be multiple columns eg. subset=['Age', 'Fare'], and will remove rows where any missing data is present in either row. Note how I'm not replacing the original Titanic dataframe and instead creating new ones from the modified original. If you want, you could modify the original by adding in the inplace=True argument to the function.

In [0]:
survived = titanic_no_age_missing['Survived'] == 1
died = titanic_no_age_missing['Survived'] == 0

t, p, df = sms.weightstats.ttest_ind(titanic_no_age_missing[survived]['Age'], titanic_no_age_missing[died]['Age'])

print(f'''
t-stat is: {t}
p-value is: {p}
degrees of freedom: {df}
''')

Lastly lets try the exact same thing with Pingouin. 

[Documentation](https://pingouin-stats.org/generated/pingouin.ttest.html)

In [0]:
# t-test using pingouin
# setup similar to statmodels

titanic_no_age_missing = titanic.dropna(subset=['Age'])

survived = titanic_no_age_missing['Survived'] == 1
died = titanic_no_age_missing['Survived'] == 0

# we set correction to False as for simplicity's sake in this example we assume the variance is the same between both groups.
results = pg.ttest(titanic_no_age_missing[survived]['Age'], titanic_no_age_missing[died]['Age'], correction=False)
results

So we used three different statistical packages in python to run the same t-test. Scipy only gave us the t-stat and p-value, while statsmodels additionally gave us the degrees of freedom. Pingouin was the most exhaustive and returned a pandas dataframe containing a number of relevant results pertaining to the t-test. Overall most of this comes down to preference.

### Your Turn

Now try and run t-tests using each of the three packages to see if the passanger's `Fare` differed between `Sex`. 

In [0]:
# Using Scipy.stats


In [0]:
# Using statsmodels


In [0]:
# Using Pingouin 


#__ANOVA__

Similar to t-tests, all the main statistical packages in Python contain their own versions of ANOVA, we won't go into detail on how to perform them with each statistical package and instead just show you one way of doing it. Though we will provode the documentation for the other methods.

[Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html)

[Statsmodels](https://www.statsmodels.org/devel/anova.html)

[Pingouin](https://pingouin-stats.org/generated/pingouin.anova.html)

Let's see if `Age` differs by the passenger's class (`Pclass`)

In [0]:
# using Scipy

# drop nans
titanic_nomiss_age = titanic.dropna(subset=['Age'])

# now filtering dataframe by ticket class
firstclass = titanic_nomiss_age['Pclass'] == 1
secondclass = titanic_nomiss_age['Pclass'] == 2
thirdclass = titanic_nomiss_age['Pclass'] == 3

# finally the ANOVA
f, p = stats.f_oneway(
    titanic_nomiss_age[firstclass]['Age'], 
    titanic_nomiss_age[secondclass]['Age'], 
    titanic_nomiss_age[thirdclass]['Age'])

print(f"""
F-value: {f}
p-value: {p}
""")

So we have found that there is a significant difference of age between the ticket classes but, as I'm sure many of you know, generally we now want to see specifically what classes are different from one another. So we need to perform post-hoc tests. One of the common post-hoc tests is Tukeys HSD and is easily performed using the statsmodels library.

In [0]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(titanic_nomiss_age['Age'], titanic_nomiss_age['Pclass'])
result = mc.tukeyhsd(alpha=0.05)  # alpha is set to 0.05 by default
print(result)

First we use the [multicomparison](https://www.statsmodels.org/stable/generated/statsmodels.sandbox.stats.multicomp.MultiComparison.html) tool, which sets the data up for multiple comparisons, and had several methods (shown at the bottom of the linked page) for different multiple comparison tests. In our case we used the [tukeyshsd](https://www.statsmodels.org/stable/generated/statsmodels.sandbox.stats.multicomp.MultiComparison.tukeyhsd.html#statsmodels.sandbox.stats.multicomp.MultiComparison.tukeyhsd) method.

##Your Turn

Try creating another ANOVA model below to see if `Fare` differs depending on where the passengers `Embarked` from. Then (even if F of group isn't significant! This is just practice) run post-hoc tests.

#__Statsmodels formulas__
Before moving into some of the slightly more complicated modeling, this is a good time to introduce one of the more powerful tools contained within the statsmodels package. For those of you that are familiar with R this will come easily (as it's based directly off of it!)

Statsmodels has the ability to use "formulas" to set up statistical models easily and efficiently, and the formula syntax is based off the R programming language. For a detailed explanation please take a look [here](https://patsy.readthedocs.io/en/latest/formulas.html#the-formula-language). 

Briefly, models can be set up using a notation like: `'y ~ x'` where `~` separates the sides of the equation this would look at how `y` is affected by `x`. By using the `+` operator you can add to one side of the equation: `'y ~ x + z'` which would look at how `y` is affected by `x` and `z`. 

To take this one step further we have the operators `:` and `*`.

`:` is a way of modeling an interaction effect: `y ~ x:z` looks at how `y` is affected by the interaction of `x` and `z`. The `*` operator is a shorthand for a full factorial model: `y ~ x*z` is the same as `y ~ x + z + x:z`.

The link provided above will go into more detail explaining the forumla syntax and how to use it, there are a few more operators we didn't go through here so I would recommend giving the page a read through.

#__MANOVA__

The easiest way of performing a MANOVA in python is to use the previously mentioned statsmodels formula syntax coupled with the statsmodels MANOVA tool.

Lets try it below:

In [0]:
from statsmodels.multivariate.manova import MANOVA

M = MANOVA.from_formula('Age + Fare ~ Pclass', data=titanic_nomiss_age)

print(M.mv_test())

#__Chi-Squared testing__

Another relatively common statistical model that is easy to implement in Python is the Chi-Squared test. The tool we will use for this is contained in the `scipy.stats` package: `chi2_contingency()`, documented [here](https://scipy.github.io/devdocs/generated/scipy.stats.chi2_contingency.html). All we need to do is format our data in the correct way to create a crosstab table. Pandas makes this very easy with their `crosstab()` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html)).

Lets try it below to look at survival rates based on sex. Our null hypotheses is that equal amounts of men and women should be present in both categories of `Survived`.

In [0]:
# creating the crosstab
ct = pd.crosstab(titanic['Sex'], titanic['Survived'])
ct

Now we have our counts let's run the test.

In [0]:
stat, p, dof, expected = stats.chi2_contingency(ct)

print(f""" 
stat: {stat} 
p: {p} 
dof: {dof}
expected:
{expected}
""")

using those results we can safely reject our null hypthesis. 

You can use pingouin to run a chi-squared as well, taking out the step of creating the crosstab. Here's a link to the [pingouin version](https://pingouin-stats.org/generated/pingouin.chi2_independence.html)

#__Linear regression__

Moving into linear regression, statsmodels again makes this relatively straightforward to perform using the formula syntax. Lets take a look at the example below looking at how `Fare` is affected by `Age`. We will be using an ordinary least squares (OLS) model through the `statsmodels.formula.OLS()` function, documented [here](https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html). 

Though we are using OLS here, statsmodels also provides other linear regression models: GLS, WLS, and GLM among others (and the parameters are all very similar to the OLS model we'll be describing here) so don't feel constrained by just using the OLS model.

In [0]:
import statsmodels.formula.api as smf

# lets first drop any missing values in our variables of interest.
titanic_nomiss_age_fare = titanic.dropna(subset=['Fare', 'Age'])

# now create the model.
model = smf.ols(formula='Fare ~ Age', data=titanic_nomiss_age_fare)

# finally we fit the model.
results = model.fit()

# and take a look at the result.
print(results.summary())

Ok that's interesting, how about we now look into additional variables. Lets add in `Sex` to the model.

In [0]:
# additionally dropping missing sex values
titanic_nomiss_age_fare_sex = titanic.dropna(subset=['Age', 'Sex', 'Fare'])

# creating new model with sex included
newmodel = smf.ols(formula='Fare ~ Age + C(Sex)', data=titanic_nomiss_age_fare_sex)

# fitting model
results = newmodel.fit()

print(results.summary())


So here we added in sex to the model, notice in the formula we surrounded it with `C()`, while in this case this may not be strictly needed, it forces statsmodels to view that variable as categorical. To finish this off let's run a model containing the interaction of `Age` and `Sex` and then run a full factorial model.

In [0]:
# just the interaction
interaction= smf.ols(formula='Fare ~ Age:C(Sex)', data=titanic_nomiss_age_fare_sex)

# fitting model
results = interaction.fit()

print(results.summary())


In [0]:
# full factorial

fullfactorial = smf.ols(formula='Fare ~ Age*C(Sex)', data=titanic_nomiss_age_fare_sex)

# fitting model
results = fullfactorial.fit()

print(results.summary())

#__Conclusion__

This was really only just a shallow dip to introduce the tools available when it comes to statistical computing with Python. There's far more than what we've presented here, and we encourage you to go out and seek new resources and tools as this really just scratches the surface. 

Again as always please contact us if you have any questions!


# Exercise #1
Go back to the annotated graphs from last week and replace `p-value = ??` with the actual p-value, rounded to 3 decimal points.

# Exercise #2
We wanted to have a fun, comprehensive exercise that will use concepts from each of the weeks.  Below will be some partially completed code that examines the differences in popularity, profit, runtime, and more between Marvel and DC movies.

In [0]:
import os, sys, re    # `re` is a package that uses regex (regular expressions). We won't go into this too much, 
# but it's an efficient way to search for strings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# load the dataset
movies = pd.read_csv('https://raw.githubusercontent.com/harshitcodes/tmdb_movie_data_analysis/master/tmdb-5000-movie-dataset/tmdb_5000_movies.csv')

In [0]:
movies.head()

Notice how some of the cells are dictionaries instead of text. The following code will use regular expressions to extract just the information we're interested in

In [0]:
genres = []   # empty list that we'll populate with the extracted values
for row in range(len(movies['genres'])):
  genre_start = [m.start()+8 for m in re.finditer('"name"', movies.genres[row])]    # list comprehension that finds each instance of '"name"' and adds 8 to get to the start of the actual value
  genres.append(', '.join([movies.genres[row][genre_start[i]: movies.genres[row].find('}', genre_start[i])] for i in range(len(genre_start))]))
  # that's a confusing line ^^
  # basically, for each instance of '"name"' we're going to collect the text from 8 characters past '"name"' until the next '}' and add that value to our genres list
  # and we're going to repeat this for each row in the dataframe

In [0]:
movies.shape[0] == len(genres)

let's do the same thing for the production companies

In [0]:
production_companies = []
for row in range(len(movies['production_companies'])):
  prod_start = [m.start()+8 for m in re.finditer('"name"', movies.production_companies[row])]
  production_companies.append(', '.join([movies.production_companies[row][prod_start[i]: movies.production_companies[row].find(',', prod_start[i])] for i in range(len(prod_start))]))
  

Replace the existing `genres` and `production_companies` series with the new lists

Make a new column in the dataframe titled `profit` and populate it with how much money each movie earned (revenue minus budget)

Filter the dataframe to only keep movies that have either `Marvel` or `DC Entertainment` listed in the production company. You can either do this at the same time, or you can make 2 new dataframes and `concatenate` them. Add a new column of either 'Marvel' or 'DC' so we can easily hue it.

Make a scatterplot of popularity by profit

Add arrows pointing to the two outliers on popularity and have the title on the other end of the arrow.

Hint: One way to get the names is to use `groupby` on the titles, filter by `popularity`, get the max value, and then sort the values.

Perform a t-test to see if popularity differs significantly by production company. Try it with and without the outliers

Perform a linear regression to see if `profit` is predicted by `popularity`, `runtime`, `production company`, and any interactions between them.